In [1]:
!pip install pandas nltk whoosh scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [40]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer
import time

from sklearn.feature_extraction.text import TfidfVectorizer

#WHOOSH
from whoosh import index
from whoosh.fields import Schema, TEXT, ID
from whoosh.analysis import StemmingAnalyzer
from whoosh.qparser import QueryParser, MultifieldParser
from whoosh.index import create_in, open_dir
from whoosh import scoring
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arashtashakori/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/arashtashakori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
#First of all we need to load the data which is in the CSV format to a Pandas dataframe to be able to access
#and manipulate it efficiently

path = "food_recipes.csv"
recipes = pd.read_csv(path)


#Cleaning the text by converting to lower case and removing punctuations: Taken from Tutorial 1
def clean_text (text):
    #Lower case the given text
    text = text.lower()

    #Removing punctuation
    text = re.sub(r'[^a-z]', ' ', text)
    return text

#Removing stopwords to focus on more meaningful words from a text: Taken from Tutorial 1
def stopwords_removal(text):
    #Getting the set of stopwords in English
    stop_words = set(stopwords.words('english'))

    #Making up the text again without stopwords
    sentence = ' '.join([word for word in text.split() if word not in stop_words])
    return sentence

#Turning every word in the given text into its more basic core word "lemma": Taken from Tutorial 1
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    sentence = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return sentence

#This is the pre-processing function that performs text cleaning, removing stop words and lemmatization and drops
#rows with empty entries
def preprocess (data):
    #While I couldn't find any empty entry it is important to drop rows with missing entries to make the code work
    #according to our Data mining class last term
    data = data.dropna()

    #Applying the preprocessing steps using the functions defined above
    data['directions'] = data['directions'].apply(clean_text)
    data['directions'] = data['directions'].apply(stopwords_removal)
    data['directions'] = data['directions'].apply(lemmatize)

    return data
    

recipes = preprocess(recipes)

#Displaying the last 5 entries of the dataframe
recipes.tail()


,title,directions
49995,Caramel Frosting,let butter melt add sugar brown add milk bring...
49996,Barbecued Chicken Wings,mix sauce brown sugar onion water mixing bowl ...
49997,Pound Cake,bake hour put toothpick fork middle done tooth...
49998,Slush,combine sugar boiling water stir cool add bana...
49999,Granny Ebert'S Hash,cover meat water teaspoon salt teaspoon pepper...


For the pre-processing step first we need to clean the data by turning all texts (descriptions) into lower case and removing all punctuations using the regular expression library of python as discussed in the tutorial. This ensures that upper case letters and punctuations that don't have a purpose in the search won't affect our search. Then we used the built-in stopwords library to get the stop words in English to remove them as they don't add any specific value for search purposes. Then we lemmatize the directions by turning every word into the base 'dictionary-like' word to unify words that have the same approximate meaning to prevent slight variations in forms of the same conceptual word affecting the search. Also, before all of that we drop all rows with empty fields as they don't have any value in our search and may cause exceptions later.

# Term-Document Incidence Matrix


PART A:

In [4]:
#For this I used CountVectorizer from sklearn's feature extraction library. I found details about using it on 
#Sklearn's website https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
#which I found by searching on Google: "Token counting library Sklearn". Please see Imports section above for the 
#importing statement.

td_matrix_maker = CountVectorizer(binary=True) #Since we are doing binary search, the exact number is irrelavant
td_incidence = td_matrix_maker.fit_transform(recipes["directions"])    #Running the CountVectorizer on the DF

#Turning the result into a dataframe of its own
incidence_matrix_lib = pd.DataFrame(td_incidence.toarray(), columns=td_matrix_maker.get_feature_names_out()).T


print(incidence_matrix_lib)


            0      1      2      3      4      5      6      7      8      \
ability         0      0      0      0      0      0      0      0      0   
able            0      0      0      0      0      0      0      0      0   
ableskiver      0      0      0      0      0      0      0      0      0   
absent          0      0      0      0      0      0      0      0      0   
absolutely      0      0      0      0      0      0      0      0      0   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
zita            0      0      0      0      0      0      0      0      0   
ziti            0      0      0      0      0      0      0      0      0   
zucchini        0      0      0      0      0      0      0      0      0   
zuchini         0      0      0      0      0      0      0      0      0   
zwieback        0      0      0      0      0      0      0      0      0   

            9      ...  49990  49991  49992  49993  49994  49995  49996  \


PART B:

In [5]:
#From tutorial 1: This is the tutorial implementation of the term_document incidence matrix creator. It finds whether a word exists in a series of entries or not. "The term incidence matrix"
def term_document_incidence_matrix(data):
    # initialize an empty set to store unique words
    words = set()
    
    # iterate over the content column in the dataframe and update the set with unique words
    for content in data:
        words.update(content.split())
    
    # convert the set to a list and sort it
    words = list(words)
    words.sort()
    
    # define an empty list to store the matrix
    matrix = []
    
    # create a row for each content in the dataframe 
    for content in data:
        
        # store the row as a list of zeros with the length of the unique words
        row = [0] * len(words)
        
        # for each word in the content, update the row with 1 where the word is found
        for word in content.split():
            row[words.index(word)] = 1
        
        matrix.append(row)
    
    # return the matrix as a dataframe with the words as columns
    return pd.DataFrame(matrix, columns=words).T

tutorial_incidence_matrix = term_document_incidence_matrix(recipes["directions"])
print(tutorial_incidence_matrix)

            0      1      2      3      4      5      6      7      8      \
ability         0      0      0      0      0      0      0      0      0   
able            0      0      0      0      0      0      0      0      0   
ableskiver      0      0      0      0      0      0      0      0      0   
absent          0      0      0      0      0      0      0      0      0   
absolutely      0      0      0      0      0      0      0      0      0   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
zita            0      0      0      0      0      0      0      0      0   
ziti            0      0      0      0      0      0      0      0      0   
zucchini        0      0      0      0      0      0      0      0      0   
zuchini         0      0      0      0      0      0      0      0      0   
zwieback        0      0      0      0      0      0      0      0      0   

            9      ...  49990  49991  49992  49993  49994  49995  49996  \


PART C:

In [6]:
print("SKLEARN LIBRARY IMPLEMENTATION:")
print (incidence_matrix_lib)
print("Tutorial Implementation:")
print(tutorial_incidence_matrix)

SKLEARN LIBRARY IMPLEMENTATION:
            0      1      2      3      4      5      6      7      8      \
ability         0      0      0      0      0      0      0      0      0   
able            0      0      0      0      0      0      0      0      0   
ableskiver      0      0      0      0      0      0      0      0      0   
absent          0      0      0      0      0      0      0      0      0   
absolutely      0      0      0      0      0      0      0      0      0   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
zita            0      0      0      0      0      0      0      0      0   
ziti            0      0      0      0      0      0      0      0      0   
zucchini        0      0      0      0      0      0      0      0      0   
zuchini         0      0      0      0      0      0      0      0      0   
zwieback        0      0      0      0      0      0      0      0      0   

            9      ...  49990  49991  49992

PART D:

In [7]:
#This method uses boolean search to find the index of rows that contains given search terms: From tutorial 1
def boolean_search(data, matrix, search_terms, operator='AND'):
    try:
        # Ensure that the search terms are in lowercase
        search_terms = [term.lower() for term in search_terms]
                
        # Filter the matrix to include only the search terms
        filtered_matrix = matrix.loc[search_terms]
    
        # Find all reviews where all terms appear (boolean AND operation)
        if operator == 'AND':
            valid_indices = filtered_matrix.columns[(filtered_matrix > 0).all()]
            
        # Find all reviews where any term appears (boolean OR operation)
        elif operator == 'OR':
            valid_indices = filtered_matrix.columns[(filtered_matrix > 0).any()]
            
        else:
            raise ValueError("Operator must be 'AND' or 'OR'")

        # Select and return the relevant data using the valid indices
        return data.loc[valid_indices, ['title', 'directions']]

    except KeyError as e:

        # Handle the case where one or more search terms are not in the matrix
        print(f"Warning: {str(e).strip('[]')} not found")
        return pd.DataFrame()  # Return an empty DataFrame if any term is not found
    

search_terms = ["sugar", "oil", "bowl", "milk"]


In [8]:
result_lib = boolean_search(recipes, incidence_matrix_lib, search_terms, operator="AND")
print(result_lib)

                              title  \
29                   One Hour Rolls   
86                      Funnel Cake   
863              Apple-Banana Bread   
1634              Golden Corn Bread   
1841    Chocolate Chip Vanilla Gems   
...                             ...   
45342              Easy Yeast Rolls   
47198               Cherry Nut Loaf   
47775        Country Fair Egg Bread   
49183  Lovelight Chiffon Layer Cake   
49260           Orange Corn Muffins   

                                              directions  
29     put flour large mixing bowl combine sugar milk...  
86     sift first ingredient together bowl mix egg mi...  
863    combine egg banana sugar oil milk separate bow...  
1634   preheat oven sift together corn meal flour sug...  
1841   preheat oven combine flour sugar baking powder...  
...                                                  ...  
45342  combine yeast warm milk soften yeast minute co...  
47198  sift flour salt soda powder large bowl mix che...  


In [9]:
result_tutorial = boolean_search(recipes, tutorial_incidence_matrix, search_terms, operator="AND")
print(result_tutorial)

                              title  \
29                   One Hour Rolls   
86                      Funnel Cake   
863              Apple-Banana Bread   
1634              Golden Corn Bread   
1841    Chocolate Chip Vanilla Gems   
...                             ...   
45342              Easy Yeast Rolls   
47198               Cherry Nut Loaf   
47775        Country Fair Egg Bread   
49183  Lovelight Chiffon Layer Cake   
49260           Orange Corn Muffins   

                                              directions  
29     put flour large mixing bowl combine sugar milk...  
86     sift first ingredient together bowl mix egg mi...  
863    combine egg banana sugar oil milk separate bow...  
1634   preheat oven sift together corn meal flour sug...  
1841   preheat oven combine flour sugar baking powder...  
...                                                  ...  
45342  combine yeast warm milk soften yeast minute co...  
47198  sift flour salt soda powder large bowl mix che...  


PART E: I am going to answer this solely based on runtime. I have isolated execution of the code for the creation of the matrices and running the search algorithm to see how long each of them takes to execute. When it comes to creating the matrix the Sklearn library's implementation of the creation of the term incidence matrix SIGNIFICANTLY outperforms the tutorial's barebone implementation. It took 1 minutes and 53 seconds for the tutorial barebone implementation to make the matrix while the sklearn library's implementation took only 0.9 second. That is the creation of the matrix with the library was 125 times faster using the library. The boolean search however was very close with the tutorial's barebone implementation taking 0.1 seconds vs sklearn's matrix taking 0.6 seconds. Still the massive difference between how long it takes to make the matrix in the first place using the tutorial's barebone implementation of the term incidence matrix, makes sklearn far more efficient for the data given for this assignment.

# Inverted Index


In [10]:
#Maps the words to document indexes and return the mapped dictionary
def map_function(data):
    mapped_dict = {}
    for index, row in data.iterrows():
        words = row['directions'].split() #Break into array of single words
    
        for word in words:
            if word in mapped_dict:    #If word is already in the dictionary append it to the appropriate index
                mapped_dict[word].append(index)    
            else:   #If not create a new index with word as key
                mapped_dict[word] = [index]
    return mapped_dict


In [11]:
#Gets the mapped dictionary from map_function and reduce it to a final inverted index where each word is associated
#with the set of document indexes in which the word appears by reducing the processed dictionary of key words and index values
def reduce_function(mapped_dict): 
    inverted_index = {}
    for word, indices in mapped_dict.items():
        if word in inverted_index:  #If the word is already in the final inverted matrix
            inverted_index[word].update(indices)    #update set of document indexes for that word
        else:   #If the word doesn't exist
            inverted_index[word] = set(indices) #Initialize a New set of indixes
    return inverted_index
# write the implementation for the map function here

In [12]:
#This just uses the last two functions to create the inverted index by feeding the mapped dictionary of words into
#the reducer function
def create_inverted_index_map_reduce(data):
    # map the data
    mapped_data = map_function(data)
    # reduce the mapped dictionary to get the inverted index
    inverted_index = reduce_function(mapped_data)
    return inverted_index

In [13]:
def inverted_index_search(recipe_data, inverted_index, search_terms):
    try:
        # Find all indices where all terms appear (boolean AND operation)
        sets_of_indices = [set(inverted_index[term]) for term in search_terms if term in inverted_index]
        if not sets_of_indices:
            return pd.DataFrame()  # Return an empty DataFrame if no indices found

        # Intersect all sets of indices to find common ones
        valid_indices = set.intersection(*sets_of_indices)

        # Convert the set of valid indices to a list
        valid_indices_list = list(valid_indices)

        # Select and return the relevant recipes using the valid indices
        return recipe_data.loc[valid_indices_list, ['title', 'directions']]
    except KeyError as e:
        # Handle the case where one or more search terms are not in the matrix
        print(f"Warning: {str(e).strip('[]')} not found in recipe terms.")
        return pd.DataFrame()  # Return an empty DataFrame if any term is not found

In [14]:
inverted_index = create_inverted_index_map_reduce(recipes)
search_results = inverted_index_search(recipes, inverted_index, search_terms)
print("Search Results:")
print(search_results)

Search Results:
                                title  \
38918          Golden Harvest Muffins   
13836        Country Fair Corn Sticks   
31762               Cinnamon Nut Cake   
43028        Baked Apple French Toast   
13333        Deep Dark Chocolate Cake   
...                               ...   
30683              Fudge Pudding Cake   
10204                 Company Muffins   
4582                   Cornbread Cake   
10732                   Pumpkin Bread   
26093  Portzeln (New Year'S Fritters)   

                                              directions  
38918  heat oven quart bowl combine flour sugar bakin...  
13836  preheat oven grease well corn stick mold place...  
31762  preheat oven mix cake mix oil egg sugar sour c...  
43028  preheat oven spray x inch baking pan dish vege...  
13333  heat oven grease flour inch baking pan large b...  
...                                                  ...  
30683  large mixing bowl stir together flour cup suga...  
10204  preheat oven

# Inverted Index using Trees

PART A:

In [15]:
# Creating the tree classes from scratch:
#The node class initializes a tree node contatining a word and its respective list of document ids
class TreeNode:
    def __init__(self, word, document_id):
        self.word = word
        self.document_ids = set([document_id])
        self.left = None
        self.right = None

#This class implements the actual binary tree BST with each left node having the word that is smaller alphabetically
class BinaryTree:
    def __init__(self):
        self.root = None

    def insert(self, word, document_id):
        if not self.root: #If the root is empty
            self.root = TreeNode(word, document_id) #Initialize the root with the node containing the word
        else:
            self._insert_rec(self.root, word, document_id)

    #Locates and puts the given word and its respective document id in its correct position or append the existing one with the id
    def _insert_rec(self, node, word, document_id):
        if word == node.word:   #If current node contains the given word
            node.document_ids.add(document_id)
        elif word < node.word:  #If word is alphabetically less than the word at current node -> go left
            if node.left is None:   #If nothing is on the left -> here it is
                node.left = TreeNode(word, document_id)
            else:   #If not, do this thing all over again recursively on the left node
                self._insert_rec(node.left, word, document_id)
        else: #If word is alphabetically more than the word at current node -> go right
            if node.right is None:  #If nothing is on the right -> here it is
                node.right = TreeNode(word, document_id)
            else:   #If not, do this thing all over again recursively on the right node
                self._insert_rec(node.right, word, document_id)

    #Returns document ids given a word or None otherwise
    def search(self, word):
        return self._search_recursively(self.root, word)

    #Recursively searches for a word to return its document ids if exist, None if not
    def _search_recursively(self, node, word):
        if node is None:   #If current node is null
            return None
        elif word == node.word: #If current node contains our word
            return node.document_ids
        elif word < node.word:  #If word is alphabetically less than the word at current node ->
            return self._search_recursively(node.left,  word)  #Pass left node to this function
        else:
            return self._search_recursively(node.right, word) #Pass right node to this function
        


#Method to read through the recipe and add its word to a single overall BTS as implemented with the classes above
def build_tree_index(data):
    tree = BinaryTree() #Initialize tree
    for index, row in data.iterrows():
        words = row["directions"].split() #Make arrays of words for each direction entry
        for word in words:
            tree.insert(word, index)
    return tree


PART B:

In [16]:
def search_tree_index(tree, query_words):
    if not query_words:
        return pd.DataFrame()  # If the query is empty, return empty df

    # Get document ids for each word in the query
    doc_sets = []
    for word in query_words:
        result = tree.search(word)  # Search the tree for the word and get the document id list back
        if result:  # If the return is not None
            doc_sets.append(result)
    
    # Intersect all document sets
    if doc_sets:
        valid_indices = set.intersection(*doc_sets)
        # exchange the set of all valid indices to a list before using it for indexing to prevent error
        valid_indices_list = list(valid_indices)
        return recipes.loc[valid_indices_list, ['title', 'directions']]
    else:
        return pd.DataFrame()  #if no words are found, return empty df


PART C:

In [17]:
#This method uses inverted index using a tree to search the dataframe for terms given as a list
def search_tree_and_print(data, search_terms):
    tree_index = build_tree_index(data)
    result = search_tree_index(tree_index, search_terms)
    print (result)


search_tree_and_print(recipes, search_terms)

                                title  \
38918          Golden Harvest Muffins   
13836        Country Fair Corn Sticks   
31762               Cinnamon Nut Cake   
43028        Baked Apple French Toast   
13333        Deep Dark Chocolate Cake   
...                               ...   
30683              Fudge Pudding Cake   
10204                 Company Muffins   
4582                   Cornbread Cake   
10732                   Pumpkin Bread   
26093  Portzeln (New Year'S Fritters)   

                                              directions  
38918  heat oven quart bowl combine flour sugar bakin...  
13836  preheat oven grease well corn stick mold place...  
31762  preheat oven mix cake mix oil egg sugar sour c...  
43028  preheat oven spray x inch baking pan dish vege...  
13333  heat oven grease flour inch baking pan large b...  
...                                                  ...  
30683  large mixing bowl stir together flour cup suga...  
10204  preheat oven combine flour o

PART D:

In [18]:
latency = []

tree_index = build_tree_index(recipes)

#Calculate the latency time 3 times and getting the averages: From Tutorial 1


start_time = time.time()
search_tree_index(tree_index, search_terms)
print("--- %s seconds ---" % (time.time() - start_time))
latency.append(time.time() - start_time)

start_time = time.time()
search_tree_index(tree_index, search_terms)
print("--- %s seconds ---" % (time.time() - start_time))
latency.append(time.time() - start_time)

start_time = time.time()
search_tree_index(tree_index, search_terms)
print("--- %s seconds ---" % (time.time() - start_time))
latency.append(time.time() - start_time)

# average latency of the times
latency = sum(latency)/len(latency)
print("\nthe average latency: ",latency)

--- 0.002095937728881836 seconds ---
--- 0.001809835433959961 seconds ---
--- 0.0015528202056884766 seconds ---

the average latency:  0.0018735726674397786


# Jaccard Similarity


PART A:

In [19]:
#This method calculates the Jaccard similarity between two sets
def jaccard_similarity(set1, set2):
    #calculating the intersection and union part of the equation to calculate the formula
    intersection_part = set1.intersection(set2)
    union_part = set1.union(set2)
    if not union_part:
        return 0  # Avoid / by zero
    similarity = len(intersection_part) / len(union_part)   #Jaccard formula
    return similarity

def jaccard_similarity_search(recipe_data, inverted_index, search_terms):
    try:
        sets_of_indices = [set(inverted_index[term]) for term in search_terms if term in inverted_index]
        if not sets_of_indices:
            return pd.DataFrame() 
        valid_indices = set.intersection(*sets_of_indices)
        valid_indices_list = list(valid_indices)
        result = recipe_data.loc[valid_indices_list, ['title', 'directions']]
        result['jaccard_similarity'] = result['directions'].apply(lambda x: jaccard_similarity(set(x.split()), set(search_terms)))

        return result
    except KeyError as e:
        print(f"Warning: {str(e).strip('[]')} not found in recipe terms.")
        return pd.DataFrame() 
    

search_terms = ["apple", "oil", "combine"]
#Please refer to above cells for where inverted_index
result = jaccard_similarity_search (recipes, inverted_index, search_terms)
print(result)

                                 title  \
5120               Fresh Apple Muffins   
18944                German Apple Cake   
35457  Parmesan Pork Chops With Apples   
18304                 Fresh Apple Cake   
38918           Golden Harvest Muffins   
...                                ...   
30711           Fresh Apple Pound Cake   
48888                 Fresh Apple Cake   
31993         Fresh Apple Or Pear Cake   
11389                 Fresh Apple Cake   
12030                 Marinated Shrimp   

                                              directions  jaccard_similarity  
5120   preheat oven large bowl mix together oil sugar...            0.066667  
18944  sift together flour baking soda salt cinnamon ...            0.120000  
35457  combine egg milk set aside combine flour parme...            0.069767  
18304  combine oil sugar blend add egg one time add v...            0.107143  
38918  heat oven quart bowl combine flour sugar bakin...            0.093750  
...                  

PART B:

In [38]:
#Calculating and printing the service latency of the Jaccard Similarity search implementation
def calc_latency(data, inverted_index, search_terms):
    latency = []


    start_time = time.time()
    jaccard_similarity_search (recipes, inverted_index, search_terms)
    print("--- %s seconds ---" % (time.time() - start_time))
    latency.append(time.time() - start_time)

    start_time = time.time()
    jaccard_similarity_search (recipes, inverted_index, search_terms)
    print("--- %s seconds ---" % (time.time() - start_time))
    latency.append(time.time() - start_time)

    start_time = time.time()
    jaccard_similarity_search (recipes, inverted_index, search_terms)
    print("--- %s seconds ---" % (time.time() - start_time))
    latency.append(time.time() - start_time)

    # average latency of the times
    latency = sum(latency)/len(latency)
    print("\nthe average latency: ",latency)

calc_latency (recipes, inverted_index, search_terms)

--- 0.06626391410827637 seconds ---
--- 0.0042951107025146484 seconds ---
--- 0.003253936767578125 seconds ---

the average latency:  0.024831612904866535


# TF-IDF

In [25]:
#For this part I am using the TfidfVectorizer library from SKlearn which I found by searching "TFIDF Vectorizer".git
#I used the following link as reference:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html 
#Please find the imports in the Imports section of Q1

#This method vectorizes a given df according to TFIDF
def make_tdidfIndex(data):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(data["directions"]) #Make up the matrix using the vectorizer
    return vectorizer, tfidf_matrix


def tfidf_similarity_search(data, query):
    vectorizer, tfidf_matrix = make_tdidfIndex(recipes)

    # Make the query a TF-IDF vector of the given corpus
    query_vector = vectorizer.transform([query])

    # make the list and  dense array
    query_arr = query_vector.toarray()
    corpus_arr = tfidf_matrix.toarray()
    
    result = []
    
    for index, doc_vector in enumerate(corpus_arr):
        # only add scores for terms that appear in the query
        similarity = sum(doc_vector[query_arr[0] > 0])
        result.append({"title": data.iloc[index]["title"], "TFIDF_similarity": similarity})

    # turn the list of dictionariess into dfs
    result_df = pd.DataFrame(result)

    #sort results by TFIDF in decreasing order
    result_df.sort_values(by="TFIDF_similarity", ascending=False, inplace=True)
    
    return result_df



query = "chocolate ice cream cake"
results = tfidf_similarity_search(recipes, query)
print(results)

                                          title  TFIDF_similarity
27255                             Ice Cream Pie          1.085778
19470                      Chocolate Milk Shake          1.069135
19701  Easy Chocolate Cake(Needs No Frosting)            1.021685
32066                       Fruit Cocktail Cake          1.018895
12717                 Strawberry Ice Cream Cake          1.014370
...                                         ...               ...
19017                  Caramel Chews(Unbaked)            0.000000
19018                  Slow-Cooked Pepper Steak          0.000000
19020                               Party Salad          0.000000
19021                Broccoli Cauliflower Salad          0.000000
49999                       Granny Ebert'S Hash          0.000000

[50000 rows x 2 columns]


PART B:

In [28]:
#Calculating and printing the service latency of the TF-IDF similarity search implementation
def calc_latency_tfidf (data, query):
    latency = []


    start_time = time.time()
    tfidf_similarity_search (data, query)
    print("--- %s seconds ---" % (time.time() - start_time))
    latency.append(time.time() - start_time)

    start_time = time.time()
    tfidf_similarity_search (data, query)
    print("--- %s seconds ---" % (time.time() - start_time))
    latency.append(time.time() - start_time)

    start_time = time.time()
    tfidf_similarity_search (data, query)
    print("--- %s seconds ---" % (time.time() - start_time))
    latency.append(time.time() - start_time)

    # average latency of the times
    latency = sum(latency)/len(latency)
    print("\nthe average latency: ",latency)

calc_latency_tfidf (recipes, query)

--- 2.604679822921753 seconds ---
--- 2.349626064300537 seconds ---
--- 2.4054667949676514 seconds ---

the average latency:  2.4536872704823813


# Search Index using Whoosh

PART A:

In [31]:
schema = Schema(
    title=TEXT(stored=True),  # Store the title to display in search results, and make it searchable
    directions=TEXT(analyzer=StemmingAnalyzer(), stored=True)  # Store and index the directions for searching
)

PART B:

In [34]:
#FROM TUTORIAL 1:
# Making sure that the index directory exists - creating it if not
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

# Create index schema and an index using the schema
index = create_in("indexdir", schema)

def write_to_index(data):
    #Create a writer object
    writer = index.writer()
    # write the data to the index
    for i, row in recipes.iterrows():
        writer.add_document(title=row["title"], directions=row["directions"])
    # Commit changes to the writer
    writer.commit()

# call the function to write to the index
write_to_index(recipes[:10000])

PART C:

In [63]:
#For this part I used Whoosh's built in Scoring class, please find it imported in the Imports section of Q1 with all
#other imports. I found information on it by searching "Whoosh score tf idf" in Google and found information on it
#from https://whoosh.readthedocs.io/en/latest/api/scoring.html 
#Also I found https://whoosh.readthedocs.io/en/latest/parsing.html and MultifieldParser on Whoosh's website. It is imported above as well

#Searches and indexes and calculates the 5 most relavant docs to query based on TF IDF scoring. If print is 1, it prints them, if 0, it only returns
#the search time
def search_index(query, to_print):
    # Open the existing index directory
    index = open_dir("indexdir")

    # create a query parser for the content field in the index schema
    query_parser = MultifieldParser(["title", "directions"], schema=index.schema)

    # Parse the query
    query = query_parser.parse(query)

    #define our TF_IDF searcher from whoosh
    searcher = index.searcher(weighting=scoring.TF_IDF())

    #THIS IS FOR PART E: calculating time for searching and indexing
    start_time = time.time()

    # search the index
    results = searcher.search(query, limit = 5)

    search_time = time.time() - start_time

    

    if (to_print != 0):
        #Printing the results
        for result in results:
            print("Title:", result["title"])
            print("Directions:", result["directions"])
            print("Score:", result.score)
            print("Search time:", search_time)
            print("\n")

    
    else:   
        return search_time

# Example usage of the search function
print("Query: \n")
print(query)
print ("\nTop 5 Results:")
search_index(query, 1)

Query: 

chocolate ice cream cake

Top 5 Results:
Title: Triple Mint Ice-Cream Angel Dessert
Directions: split cake horizontally four equal layer place bottom cake layer serving plate spread top layer half chocolate mint ice cream within inch edge top second cake layer cover freeze minute firm spread second cake layer pink peppermint ice cream add third cake layer cover freeze minute firm spread third cake layer remaining chocolate mint ice cream top remaining cake layer freeze firm
Score: 68.36586268081896
Search time: 0.047796010971069336


Title: Ice Cream Cake
Directions: layer crushed oreo bottom oblong dish pan lace chocolate syrup add layer chocolate ice cream sprinkle crushed oreo ice cream layer add layer vanilla ice cream cover cool whip decorate sprinkle syrup oreo
Score: 44.836604949704785
Search time: 0.047796010971069336


Title: Chocolate Devil'S Food Cake
Directions: cream crisco sugar add egg sift add flour spice together salt dissolve chocolate hot coffee soda butterm

PART C Justification for using TF-IDF: TF-IDF is ideal for finding the score of the importance of a query in a document as it both has the term frequency part which assesses the number of occurences of the terms within the document and also it has the Inverse document frequency which assesses the informativeness of the term within the document which in a sense normalizes the score given to the document based on the query. This makes it ideal as it does not allow certain not-so-informative terms that may occur a lot dominate the scoring as it normalizes their importance based on the IDF, while also taking the number of occurrences into account.

PART D: ANSWERED IN PART C (See above). I just do a few queries for this part.

In [73]:
query2 = "sugar oil bowl milk"
print("Query: \n")
print(query2)
print ("\nTop 5 Results:")
search_index(query2, 1)

query3 = "cheese burger"
print("Query: \n")
print(query3)
print ("\nTop 5 Results:")
search_index(query3, 1)

Query: 

sugar oil bowl milk

Top 5 Results:
Title: Cinnamon Rolls
Directions: scald milk combine sugar oil salt bowl add milk stir dissolve cool lukewarm soften yeast water add milk mixture beat egg add milk mix add flour mix soft dough knead minute put greased bowl rise warm place hour roll oblong inch thick brush tablespoon sugar teaspoon cinnamon roll cut inch slice lay flat greased round pan rise hour bake minute
Score: 24.945647150817244
Search time: 0.0771479606628418


Title: Doughnuts
Directions: soften yeast water add teaspoon cup sugar set aside add fat shortening rest sugar salt hot milk stir sugar dissolved cool add egg stir softened yeast stir flour liquid ingredient well mixed turn dough onto lightly floured board knead quickly smooth elastic form smooth ball place greased bowl turn grease surface cover let rise warm place double bulk hour turn onto board knead well roll cut doughnut cutter place waxed paper let rise double bulk cook cooking oil brown still hot roll powd

PART E: Implemented part E.a in PART C's function. Averaging:

In [74]:
#Taking average of 10 runs
total_time = 0
query4 = "sugar oil bowl milk"
for i in range(10):
    total_time = total_time + search_index(query4, 0)

print(total_time/10)

0.07534978389739991


# Performance Discussion

PART A: The average service latency for the Inverted index using trees was 0.0018735726674397786, Jaccard Similarity was 0.024831612904866535, TF-IDF without using cosine similarity was 2.4536872704823813, and the average service latency of Whoosh using TF-IDF was 0.07534978389739991. This makes Inverted Index using trees the fastest by far followed by Jaccard Similarity and Whoosh. All methods were given the same dataframe and the result was calculated as an average of multiple runs. This is a fairly reliable method as it is the same method of testing averaged over multiple time frames. I believe the reason behind the inverted index being faster for the purpose of this dataframe is that after creating a tree it is very efficient and to traverse the tree to find the specific word. But it does need an initial setting up of the tree which could be time consuming.

Most of the search methods returned the exact same thing, some of them just returned a limited amount of result which could be more helpful in certain situations like in implementing search engines but generally the detected results were similar. TF-IDF methods returned better results as they take both the relevance and importance of each word into account.